<a href="https://colab.research.google.com/github/dajoeng/KDT_team4_FinalPJT/blob/main/01_%EC%9B%B9%ED%81%AC%EB%A1%A4%EB%A7%81/%EB%A7%8C%EA%B0%9C%EC%9D%98%EB%A0%88%EC%8B%9C%ED%94%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive

Mounted at /content/drive
/content/drive/My Drive


In [2]:
import pandas as pd
import re
import numpy as np
import itertools

In [19]:
header= ["레시피일련번호",'레시피제목','요리명','등록자ID','등록자명','조회수','추천수','스크랩수','요리방법별명','요리상황별명','요리재료별명','요리종류별명','요리소개','요리재료내용','요리인분명','요리난이도명',"요리시간명","최초등록일시"]
ckg = pd.read_csv('만개의레시피.csv', header=None, names=header, encoding='cp949')
ckg.drop(0, inplace=True)
ckg.info()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,5,6,7,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 128400 entries, 1 to 128400
Data columns (total 18 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   레시피일련번호  128400 non-null  object
 1   레시피제목    128400 non-null  object
 2   요리명      126310 non-null  object
 3   등록자ID    128400 non-null  object
 4   등록자명     128394 non-null  object
 5   조회수      128400 non-null  object
 6   추천수      128400 non-null  object
 7   스크랩수     128400 non-null  object
 8   요리방법별명   128400 non-null  object
 9   요리상황별명   127510 non-null  object
 10  요리재료별명   128400 non-null  object
 11  요리종류별명   128400 non-null  object
 12  요리소개     127783 non-null  object
 13  요리재료내용   127340 non-null  object
 14  요리인분명    124171 non-null  object
 15  요리난이도명   128137 non-null  object
 16  요리시간명    120054 non-null  object
 17  최초등록일시   128400 non-null  object
dtypes: object(18)
memory usage: 18.6+ MB


In [20]:
ckg.drop(['등록자ID','등록자명','요리인분명','요리난이도명',"요리시간명","최초등록일시"], axis=1, inplace=True)
ckg.head()

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용
1,128671,어묵김말이,어묵김말이,9592,6,66,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...
2,128892,두부에 꼬리가 달렸어요!!,NaN,5538,3,26,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...
3,128932,입안에서 톡톡톡,NaN,6802,8,36,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...
4,131871,★현미호두죽,현미호두죽,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간
5,139247,부들부들 보들보들 북어갈비♥,북어갈비,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...


In [23]:
#요리재료내용 결측치 삭제
ckg_drop_index = ckg[ckg.loc[:,'요리명'].isnull()].index
ckg_drop = ckg.drop(ckg_drop_index)
ckg_drop_index = ckg_drop[ckg_drop.loc[:,'요리재료내용'].isnull()].index
ckg_drop = ckg_drop.drop(ckg_drop_index)
ckg_drop.reset_index(drop=True, inplace=True)
ckg_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125389 entries, 0 to 125388
Data columns (total 12 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   레시피일련번호  125389 non-null  object
 1   레시피제목    125389 non-null  object
 2   요리명      125389 non-null  object
 3   조회수      125389 non-null  object
 4   추천수      125389 non-null  object
 5   스크랩수     125389 non-null  object
 6   요리방법별명   125389 non-null  object
 7   요리상황별명   124528 non-null  object
 8   요리재료별명   125389 non-null  object
 9   요리종류별명   125389 non-null  object
 10  요리소개     124921 non-null  object
 11  요리재료내용   125389 non-null  object
dtypes: object(12)
memory usage: 11.5+ MB


In [24]:
#디저트, 양념/소스/잼, 과자, 빵 카테고리 제외
drop_list = ckg_drop.loc[(ckg_drop['요리종류별명']=='디저트')
                        | (ckg_drop['요리종류별명']=='양념/소스/잼')
                        | (ckg_drop['요리종류별명']=='빵')
                        | (ckg_drop['요리종류별명']=='과자')
                        | (ckg_drop['요리종류별명']=='샐러드')
                        | (ckg_drop['요리종류별명']=='스프')].index
ckg_drop.drop(drop_list, inplace=True)
ckg_drop.reset_index(drop=True, inplace=True)
ckg_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98691 entries, 0 to 98690
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   레시피일련번호  98691 non-null  object
 1   레시피제목    98691 non-null  object
 2   요리명      98691 non-null  object
 3   조회수      98691 non-null  object
 4   추천수      98691 non-null  object
 5   스크랩수     98691 non-null  object
 6   요리방법별명   98691 non-null  object
 7   요리상황별명   98216 non-null  object
 8   요리재료별명   98691 non-null  object
 9   요리종류별명   98691 non-null  object
 10  요리소개     98335 non-null  object
 11  요리재료내용   98691 non-null  object
dtypes: object(12)
memory usage: 9.0+ MB


In [25]:
ckg_drop.head()

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용
0,131871,★현미호두죽,현미호두죽,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간
1,139247,부들부들 보들보들 북어갈비♥,북어갈비,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...
2,149207,토마토스파게티♥,토마토스파게티,12754,2,36,볶음,일상,가공식품류,면/만두,한번 만들어 두면 이모저모 쓸 용도가 많은 토마토소스를 이용한 토마토 스파게티.,[재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...
3,151148,표고버섯탕수,표고버섯탕수,16053,2,194,튀김,손님접대,버섯류,메인반찬,탕수육 먹을때 칼로리때문에 걱정하시는 분들!! 이제는 걱걱걱 정하지 마세요~ 칼로리...,[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...
4,153040,달콤한 마늘향이 가득해~ 갈릭치킨♥,갈릭치킨,8148,0,38,튀김,손님접대,닭고기,메인반찬,요즘 갈릭이 대세인거 같아요~ 그래서 오늘은 제가 너무 좋아하는 닭과 마늘을 한번에...,[재료] 닭 1마리 [닭양념] 통마늘 3개| 다진마늘 1T| 화이트와인또는우유| 올...


## 요리 재료 전처리

In [26]:
###재료 전처리

#요리재료_전처리(new column)
ckg_drop['요리재료_전처리'] = np.NaN
one_len = ['무','파','꿀','깨','떡','햄','콩','빵','닭','밤','면','잣','쌈', '마', '쑥','굴']

for idx in range(len(ckg_drop)):
    ingredient = ckg_drop.loc[idx, '요리재료내용']
    ingredient = re.sub(r'\[(.*?)\]', '|', ingredient)
    ingredient = re.sub(r'\([^)]*\)|[^ㄱ-ㅣ가-힣 \|]', ' ', str(ingredient)) #한국어 제외 모두 제거

    ingredient = ingredient.strip()
    ingredients = ingredient.split('|')

    ingre_lists = []
    for i in ingredients:
        i = i.strip()
        i = i.split(' ')

        if (len(i) == 1) and (i != None) : #계량 단위 없는 경우
            ingre_lists.append(i)
        elif len(i) > 1:
            i.pop(-1)    #가장 마지막의 계량 단위 제거
            i = ' '.join(i)
            i = [i.strip()]
            ingre_lists.append(i)
        else :
            pass
    
    # #한자리 요리 재료 제외한 나머지 제거 ## 아직 보류
    # ingredient_list = []
    # for ingredient in ingredients:
    #     if (len(i) == 1) and (i not in one_len):
    #             pass
    #     else:
    #         ingredient_list.extend(i)

    #차원 축소
    ingre_lists = list(itertools.chain(*ingre_lists))
    
    #null 제거
    ingre_lists = list(filter(None, ingre_lists))
    
    #요리재료_전처리(new column)에 추가
    ckg_drop.loc[idx, '요리재료_전처리'] = ', '.join(ingre_lists)

ckg_drop.iloc[10000:10005,:]

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리
10000,5069915,색다른 보양식 - 전복초,전복초,24070,10,628,조림,영양식,해물류,메인반찬,전복은 어떻게 먹어도 맛있지만 오늘은 간장에 간간하게 조려낸 전복초를 만들었어요. ...,[재료] 전복 4마리| 밤| 호박씨| 홍고추| 쪽파 [양념] 간장 4TS| 물 4T...,"전복, 밤, 호박씨, 홍고추, 쪽파, 간장, 물, 마늘, 흑설탕"
10001,5070453,닭볶음탕 맛있게 만드는법,닭볶음탕,20681,8,317,끓이기,영양식,닭고기,국/탕,감자와 양파등을 넣어 골라먹는재미도있고..칼칼한 매운맛이 자꾸만 손이가는 닭볶음탕이...,[재료] 닭 1마리| 닭볶음탕 양념 1봉지| 감자(중)2개| 당근1/2개| 양파1개...,"닭, 닭볶음탕 양념, 감자, 당근, 양파, 청고추, 홍고추, 대파, 생수"
10002,5071805,멜론스무디,멜론스무디,18513,12,298,기타,일상,과일류,차/음료/술,시원하고 달콤한 멜론스무디 소개합니다.,[재료] 멜론 210g| 우유 100ml| 꿀 1작은술,"멜론, 우유, 꿀"
10003,5072495,왕고기전 만들기,고기전,30928,10,541,부침,일상,육류,밑반찬,고기와 야채 듬뿍 넣어 푸짐하고 부드러운 고기전입니다.,[재료] 다진소고기| 파프리카(빨강노랑)| 양파| 계란| 밀가루 [소고기 양념] 굴...,"다진소고기, 파프리카, 양파, 계란, 밀가루, 굴소스, 소갈비양념, 마늘, 미림, ..."
10004,5072543,삼색 감자채 볶음,삼색감자채볶음,16337,5,269,볶음,일상,채소류,밑반찬,감자에는 전분 지방 탄수화물이 많고 단백질이나 비타민이 적을 것이라고 오해들 하...,[재료] 감자 5개| 풋고추 2개| 당근 50g [양념] 소금 조금| 올리브유 1큰...,"감자, 풋고추, 당근, 소금, 올리브유, 물, 검은깨"


# 만개의 레시피에서 조회수 기반 스크랩수 가중치 계산

In [ ]:
# 원본 데이터에 에러가 존재
# csv로 저장 후 다시 불러오는 식으로 에러 해결
ckg_drop.to_csv('ckg_drop.csv')

In [44]:
ckg_drop = pd.read_csv('ckg_drop.csv')
ckg_drop.drop('Unnamed: 0', inplace=True, axis=1)

#요리재료 전처리 후, 결측치 제거
ckg_drop.dropna(inplace=True)
ckg_drop.reset_index(inplace=True, drop=True)
ckg_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97860 entries, 0 to 97859
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   레시피일련번호   97860 non-null  int64 
 1   레시피제목     97860 non-null  object
 2   요리명       97860 non-null  object
 3   조회수       97860 non-null  int64 
 4   추천수       97860 non-null  int64 
 5   스크랩수      97860 non-null  int64 
 6   요리방법별명    97860 non-null  object
 7   요리상황별명    97860 non-null  object
 8   요리재료별명    97860 non-null  object
 9   요리종류별명    97860 non-null  object
 10  요리소개      97860 non-null  object
 11  요리재료내용    97860 non-null  object
 12  요리재료_전처리  97860 non-null  object
dtypes: int64(4), object(9)
memory usage: 9.7+ MB


In [27]:
def weighted_rating(v,m,R,C):
    '''
    Calculate the weighted rating

    ( (각 조회수 / (각 조회수+최소조회수)) * 각 스크랩수 ) + ( (최소조회수/(각 조회수+최소조회수)) * 평균스크랩수 )
    
    Returns: pd.Series
    '''
    return ( (v / (v + m)) * R) + ( (m / (v + m)) * C )

In [45]:
# calcuate input parameters
C = np.mean(ckg_drop['스크랩수'])
m = np.percentile(ckg_drop['조회수'], 70)
vote_count = ckg_drop[ckg_drop['조회수'] >= m]
R = ckg_drop['스크랩수']
v = ckg_drop['조회수']

ckg_drop['weighted_clipping'] = weighted_rating(v,m,R,C)
ckg_drop.to_csv('ckg_weighted.csv') #가중치 추가 저장
ckg_drop.head()

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
0,131871,★현미호두죽,현미호두죽,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,"현미, 찹쌀, 호두, 물, 소금",314.655903
1,139247,부들부들 보들보들 북어갈비♥,북어갈비,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,"북어포, 찹쌀가루, 간장, 설탕, 물, 다진파, 다진마늘, 참기름, 깨소금, 후춧가루",284.844463
2,149207,토마토스파게티♥,토마토스파게티,12754,2,36,볶음,일상,가공식품류,면/만두,한번 만들어 두면 이모저모 쓸 용도가 많은 토마토소스를 이용한 토마토 스파게티.,[재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...,"파스타면, 토마토, 토마토, 양파, 다진마늘, 피망, 올리브유",212.802388
3,151148,표고버섯탕수,표고버섯탕수,16053,2,194,튀김,손님접대,버섯류,메인반찬,탕수육 먹을때 칼로리때문에 걱정하시는 분들!! 이제는 걱걱걱 정하지 마세요~ 칼로리...,[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...,"건표고버섯, 오이, 당근, 양파, 사과, 그외의, 과일, 녹말가루, 물, 계란 노른...",279.114241
4,153040,달콤한 마늘향이 가득해~ 갈릭치킨♥,갈릭치킨,8148,0,38,튀김,손님접대,닭고기,메인반찬,요즘 갈릭이 대세인거 같아요~ 그래서 오늘은 제가 너무 좋아하는 닭과 마늘을 한번에...,[재료] 닭 1마리 [닭양념] 통마늘 3개| 다진마늘 1T| 화이트와인또는우유| 올...,"닭, 통마늘, 다진마늘, 화이트와인또는우유, 올리브유또는버터, 마늘솔트또는소금, 파...",251.350570


## 재료 전체 사전을 만들기 위한 작업

In [54]:
ingredient_list = []
for ingredient in ckg_drop['요리재료_전처리']:
    ingredients = ingredient.split(', ')
    ingredient_list.extend(ingredients)
len(ingredient_list)

932325

In [ ]:
# #한자리 요리 재료 제외한 나머지 제거
# one_len = ['무','파','꿀','깨','떡','햄','콩','빵','닭','밤','면','잣','쌈', '마', '쑥','굴']

# ingredient_list = []
# for ingredient in ckg_drop['요리재료_전처리']:
#     ingredients = ingredient.split(', ')
    
#     for i in ingredients:
#         if (len(i) == 1) and (i not in one_len):
#             pass
#         else:
#             ingredient_list.extend(i)


In [73]:
#쿠팡 밀키트에만 있는 재료 추가
coupang_only = ['샤브용육수', '숙면','전골용 육수','곱창비법','오뗄팜S','소 스지', '전각 슬라이스','해물탕 양념',
                '유자폰즈','소간장소스', '샤브용육수','유탕면','돼지고기 민찌','얼큰짬뽕소스', '오징어건더기',
                '명태곤이','된장국양념','꽃게된장찌개용 양념', '바지락맛 액상소스','해물탕 양념','유탕면','조피볼락',
                '진종합', '순살가마볼', '고추볼', '한모듬가마보꼬', '순살가마보꼬', '간사이원형', '어묵만두','가마보꼬바',
                '프레스 햄','짬뽕 소스','쥬키니 호박','야채 믹스','수비드찜닭','사골농축액','사골 농축액','찹햄',
                '코테기노','파개장 오일', '파개장 소스','문치가자미','문치가자미','스테이크 씨즈닝','돈마호크','오향보쌈',
                '보쌈무','대두유','우동 면','올리브 오일','크러쉬드 페퍼', '사천 고추', '허브 솔트','훈제 파프리카 파우더',
                '감자 옹심이','해물 엑기스','두반장 소스','단호박 칼국수','양장피 면','새꼬막살','수비드 찜닭']
ingredient_list.extend(coupang_only)

In [74]:
#중복값 제거
ingredient_list = list(filter(None, ingredient_list))
ingredient_set = list(set(ingredient_list))
print(len(ingredient_set),ingredient_set[:20])

38482 ['무침 소금', '맛술', '핏물제거용물', '쌀눈유', '멸치다시마육수 티백', '냉이  줌', '김가루 또는', '냉 육수', '중간굵기', '훈제오리큰거', '대하머리', '초간장 소스', '시럽용올리고당', '쌀밥 한', '작두 콩', '아마씨오일', '손질된닭', '달랑 무', '모둠견과류', '불고기용 돼지뒷다리살']


In [75]:
print([i for i in ingredient_set if len(i) == 1])

['피', '채', '중', '윙', '딱', '돌', '기', '황', '찐', '쌈', '술', '국', '당', '팬', '암', '삼', '빵', '햇', '쪽', '팩', '칠', '쇠', '갈', '봄', '다', '곰', '곤', '긴', '묵', '떡', '백', '발', '롱', '복', '걀', '썬', '우', '산', '홍', '단', '한', '적', '찬', '반', '붉', '릭', '절', '진', '봉', '솥', '쨈', '뉴', '뼈', '팥', '칡', '컵', '노', '캐', '약', '생', '깨', '배', '박', '글', '순', '청', '찰', '차', '풋', '조', '덕', '실', '김', '용', '왕', '물', '잔', '도', '꿀', '풀', '게', '줌', '통', '신', '포', '건', '꽃', '원', '무', '숯', '웍', '들', '파', '잼', '소', '곡', '쌀', '유', '꼼', '취', '맛', '활', '굴', '퍼', '캔', '혼', '회', '밥', '목', '호', '된', '칼', '큰', '밀', '땅', '초', '깐', '장', '날', '립', '귤', '참', '랩', '케', '티', '논', '마', '흑', '흰', '알', '양', '돔', '매', '뮬', '간', '햄', '체', '몰', '감', '톳', '손', '찜', '쏙', '탕', '전', '핫', '루', '갓', '판', '밤', '근', '솔', '애', '쑥', '잣', '딜', '꿩', '집', '믈', '럼', '홀', '망', '콩', '콘', '부', '섭', '빨', '분', '병', '개', '엿', '연', '닭', '살', '밑', '면', '세', '골', '잎', '새']


In [76]:
#텍스트 파일로 전체 재료 저장(중복x)
with open('만개의레시피_재료_목록.txt', 'w') as f :
    for ingredient in ingredient_set:
        f.write(ingredient+',')
len(ingredient_set)

38482

In [77]:
def countLetters(word):
    counter = {}
    for letter in word:
        if letter not in counter:
            counter[letter]=0
        counter[letter] +=1
    return counter

In [78]:
# 오타검수를 위해 카운트 후 pickle 저장
ingredient_list2 = countLetters(ingredient_list)

import pickle
with open('만개의레시피_재료_목록_오타처리용.pickle', 'wb') as fw :
    pickle.dump(ingredient_list2, fw)
len(ingredient_list2)

38482